Установка зависимостей.

In [ ]:
!sudo apt install xvfb

In [ ]:
!pip install gymnasium[atari,accept-rom-license]
!pip install stable-baselines3
!pip install PyVirtualDisplay

Проверяем что скачали

In [ ]:
from stable_baselines3 import DQN, A2C

In [ ]:
import gymnasium as gym
import ale_py
print('gym:', gym.__version__)
print('ale_py:', ale_py.__version__)

gym.envs.registration.registry.keys()

Инициализируем среду

In [ ]:
env = gym.make("ALE/Pong-v5", difficulty=2, mode=0, render_mode='rgb_array')
env.unwrapped.get_action_meanings()

... и ее пространства

In [ ]:
observation_space = env.observation_space
action_space = env.action_space

In [ ]:
model = DQN("MlpPolicy", env, verbose=1)
timestamp=100
model.learn(total_timesteps=timestamp, log_interval=4)
model.save("dqn_pong")

In [ ]:
import gymnasium as gym # кто вставил здесь старый gym, уууть
from IPython import display
from pyvirtualdisplay import Display
import matplotlib.pyplot as plt
from matplotlib import animation


d = Display()
d.start()

env = gym.make("ALE/Pong-v5",  render_mode='rgb_array', difficulty=1, mode=0)
obs, info = env.reset()

img = []
for _ in range(200):
    action, _states = model.predict(obs, deterministic=True)
    obs, reward, terminated, truncated, _ = env.step(action)

    # display.clear_output(wait=True)
    img.append(env.render())

    if terminated or truncated:
        env.reset()

env.close()
dpi = 72
interval = 50 # ms

plt.figure(figsize=(img[0].shape[1]/dpi,img[0].shape[0]/dpi),dpi=dpi)
patch = plt.imshow(img[0])
plt.axis=('off')
animate = lambda i: patch.set_data(img[i])
ani = animation.FuncAnimation(plt.gcf(),animate,frames=len(img),interval=interval)
display.display(display.HTML(ani.to_jshtml()))